In [1]:
#TERCEIRA IMPLEMENTAÇÃO
#BIBLIOTECA TIME (ANALISAR O TEMPO DE EXECUÇÃO DO SOLVER)
#FUNÇÃO TIMELIMIT DO GUROBI PARA LIMITAR O TEMPO DE EXECUÇÃO
#INSTANCIAS GERADAS PARA TESTES
# 499 CLIENTES E 1 DEPOSITO
# DEMANDAS DE COLETA E ENTREGA GERADAS ALEATORIAMENTE (ENTRE 1 E 10)
# MATRIZ DE DISTANCIA GERADA ALEATORIAMENTE (ENTRE 10 E 30)
# TIMELIMIT 1600 SEGUNDOS

In [22]:
import gurobipy as gp
import time
import pandas as pd
import numpy as np
import datetime


In [14]:
V = list(range(1, 500))
V_0 = [0] + V
n = len(V)  # numero de clientes
Q = 500  # capacidade
MD = 2000  # Distancia Maxima para todo k
K = 5  # numero de veiculos disponiveis


In [29]:
matrizz = pd.read_csv('../dados/Dados Gerados (testes)/499Pontos_e_1Deposito/distancia.csv', header=None)
demandass = pd.read_csv("../dados/Dados Gerados (testes)/499Pontos_e_1Deposito/demanda.csv", header=None)


In [16]:
Distance_Matrix = matrizz.to_numpy()
Demandas = {}
Demandas['coleta'] = demandass[0].tolist()
Demandas['entregas'] = demandass[1].tolist()
veiculos = [i + 1 for i in range(K)]


In [17]:
m = gp.Model("VRPSPD")


Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-17


In [18]:
# variaveis
x = m.addVars(V_0, V_0, veiculos, vtype=gp.GRB.BINARY, name="x")
y = m.addVars(V_0, V_0, vtype=gp.GRB.INTEGER, name="y")  # coleta
z = m.addVars(V_0, V_0, vtype=gp.GRB.INTEGER, name="z")  # entrega


In [19]:
obj = gp.LinExpr()
for k in veiculos:
    for i in V_0:
        for j in V_0:
            obj += Distance_Matrix[i][j] * x[i, j, k]


In [20]:
# funcao objetivo
m.setObjective(obj, sense=gp.GRB.MINIMIZE)


In [21]:
# restrições
r1 = m.addConstrs(
    (gp.quicksum(x[i, j, k] for i in V_0 for k in veiculos) == 1 for j in V), name="X_ijk_eq_1"
)
r2 = m.addConstrs(
    (
        (gp.quicksum(x[i, j, k] for i in V_0)) -
        (gp.quicksum(x[j, i, k] for i in V_0)) == 0
        for j in V_0
        for k in veiculos
    ), name="chega_e_sai"
)
r3 = m.addConstrs(
    (
        gp.quicksum(x[0, j, k] for j in V) <= 1
        for k in veiculos
    ),
    name="max_k_veiculos"
)
r4 = m.addConstrs(
    (
        gp.quicksum(Distance_Matrix[i][j]*x[i, j, k]
                    for i in V_0 for j in V_0) <= MD
        for k in veiculos
    ),
    name="distancia_max_k"
)
r5 = m.addConstrs(
    (
        gp.quicksum(y[j, i] for i in V_0) - gp.quicksum(y[i, j] for i in V_0)
        == Demandas['coleta'][j] for j in V_0 if j != 0
    ),
    name="fluxo_y"
)
r6 = m.addConstrs(
    (
        gp.quicksum(z[i, j] for i in V_0) - gp.quicksum(z[j, i] for i in V_0)
        == Demandas['entregas'][j] for j in V_0 if j != 0
    ),
    name="fluxo_z"
)
r7 = m.addConstrs(
    (
        y[i, j] + z[i, j] <= Q*gp.quicksum(x[i, j, k] for k in veiculos)
        for i in V_0
        for j in V_0
    ),
    name="capacidade"
)


In [24]:
m.setParam('TimeLimit', 60)  # 60 segundos
m.update()
data_atual = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
m.write(f'../log/{data_atual}_VRPSPD.lp')


In [25]:

start_time = time.time()
m.optimize()
end_time = time.time()


Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 254007 rows, 1750000 columns and 7738499 nonzeros
Model fingerprint: 0x3f064e04
Variable types: 0 continuous, 1750000 integer (1250000 binary)
Coefficient statistics:
  Matrix range     [3e-02, 5e+02]
  Objective range  [3e-02, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 500 rows and 3500 columns (presolve time = 5s) ...
Presolve removed 500 rows and 3500 columns (presolve time = 10s) ...
Presolve removed 505 rows and 3500 columns (presolve time = 15s) ...
Presolve removed 500 rows and 3495 columns
Presolve time: 19.24s
Presolved: 253507 rows, 1746505 columns, 7728249 nonzeros
Variable types: 0 continuous, 1746505 integer (1247505 binary)
Deterministic concurrent LP optimizer: primal simp

In [26]:
total_time = m.getAttr("Runtime")
print("Tempo total de execução do modelo: ", total_time)
print("Tempo real de execução: ", end_time - start_time)


Tempo total de execução do modelo:  60.45099997520447
Tempo real de execução:  60.45389485359192


Tempo limite atingido. Melhor solução encontrada até agora:  inf
